# Maestría en "Analítica de datos"
---
Nombre: Jader Antonio Gomez Orrego
Código: 1018422976
Fecha: 2025 agosto 28
---
Descripción: este libro trabaja ejercicios de "analítica/ETL/estadística descriptiva"

# 1.Cargar librerias

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2.carga/obtención de **información**

In [4]:
file_path_csv="/content/drive/MyDrive/BigData/Dataset's/Población_BDUA_Regimen/Población_Base_de_Datos_Única_de_Afiliados_BDUA_del_régimen_contributivo_20250922.csv"
try:
  df=pd.read_csv(file_path_csv)
except FileNotFoundError:
  print(f"El archivo {file_path_csv} no fue encontrado.")
except Exception as e:
  print(f"Ocurrió un error al leer el archivo: {e}")

In [5]:
pd.set_option("display.max_columns", None)
display(df.head())

,Género,Grupo etario,Código de la entidad,Nombre de la entidad,Régimen,Tipo de afiliado,Estado del afiliado,Condición del beneficiario,Zona de Afiliación,Departamento,Municipio,tps_nvl_ssb_id,Nivel del Sisbén,Cantidad de registros
0,Femenino,1 a 5,EPS037,NUEVA EPS S.A.,Contributivo,BENEFICIARIO,Activo,NO APLICA,Urbana,ANTIOQUIA,BETULIA,N,VÍCTIMAS DEL CONFLICTO ARMADO INTERNO,2
1,Masculino,15 a 19,EPS041,NUEVA EPS S.A. -CM,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,HUILA,PITALITO,1,POBLACIÓN CON SISBEN,2
2,Femenino,15 a 19,EPS041,NUEVA EPS S.A. -CM,Contributivo,BENEFICIARIO,Activo,NO APLICA,Urbana,BOYACA,SOGAMOSO,NaN,NaN,13
3,Femenino,50 a 55,EPS037,NUEVA EPS S.A.,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,CORDOBA,LA APARTADA,NaN,NaN,3
4,Femenino,60 a 65,EPS037,NUEVA EPS S.A.,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,CUNDINAMARCA,ARBELAEZ,1,POBLACIÓN CON SISBEN,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641321 entries, 0 to 641320
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Género                      641321 non-null  object
 1   Grupo etario                641321 non-null  object
 2   Código de la entidad        641321 non-null  object
 3   Nombre de la entidad        641321 non-null  object
 4   Régimen                     641321 non-null  object
 5   Tipo de afiliado            641321 non-null  object
 6   Estado del afiliado         641321 non-null  object
 7   Condición del beneficiario  641321 non-null  object
 8   Zona de Afiliación          641321 non-null  object
 9   Departamento                641321 non-null  object
 10  Municipio                   641321 non-null  object
 11  tps_nvl_ssb_id              360387 non-null  object
 12  Nivel del Sisbén            360313 non-null  object
 13  Cantidad de registros       6

In [7]:
df.describe()

,Género,Grupo etario,Código de la entidad,Nombre de la entidad,Régimen,Tipo de afiliado,Estado del afiliado,Condición del beneficiario,Zona de Afiliación,Departamento,Municipio,tps_nvl_ssb_id,Nivel del Sisbén,Cantidad de registros
count,641321,641321,641321,641321,641321,641321,641321,641321,641321,641321,641321,360387,360313,641321
unique,2,12,31,31,1,3,2,3,6,33,1035,4,19,3023
top,Masculino,19 a 45,EPS037,NUEVA EPS S.A.,Contributivo,COTIZANTE,Activo,NO APLICA,Urbana,ANTIOQUIA,SANTIAGO DE CALI,1,POBLACIÓN CON SISBEN,1
freq,324586,151690,272932,272932,641321,329677,594972,612965,308602,80836,6093,139814,238525,278471


# 3.ETL

In [8]:
# Convert 'Cantidad de registros' to numeric, coercing errors, then to integer
df['Cantidad de registros'] = pd.to_numeric(df['Cantidad de registros'], errors='coerce')
df['Cantidad de registros'] = df['Cantidad de registros'].fillna(0).astype(int)
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641321 entries, 0 to 641320
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Género                      641321 non-null  object
 1   Grupo etario                641321 non-null  object
 2   Código de la entidad        641321 non-null  object
 3   Nombre de la entidad        641321 non-null  object
 4   Régimen                     641321 non-null  object
 5   Tipo de afiliado            641321 non-null  object
 6   Estado del afiliado         641321 non-null  object
 7   Condición del beneficiario  641321 non-null  object
 8   Zona de Afiliación          641321 non-null  object
 9   Departamento                641321 non-null  object
 10  Municipio                   641321 non-null  object
 11  tps_nvl_ssb_id              360387 non-null  object
 12  Nivel del Sisbén            360313 non-null  object
 13  Cantidad de registros       6

None

In [9]:
columnas_a_convertir =["Departamento","Municipio","Código de la entidad","Nombre de la entidad","Nivel del Sisbén"]
for columna in columnas_a_convertir:
  if columna in df.columns:
    df[columna]=df[columna].astype(str)


#4.estadísticas descriptivas

In [10]:
df['Género'].value_counts()

,count
Género,
Masculino,324586
Femenino,316735


In [11]:
df_eps= df.groupby('Nombre de la entidad')['Cantidad de registros'].sum()
#calcular porcentaje
total_registros = df['Cantidad de registros'].sum()
entidad_porcentaje = (df_eps / total_registros) * 100
resultado_df = pd.DataFrame({'Cantidad de Registros': df_eps, 'Porcentaje': entidad_porcentaje})
resultado_df = resultado_df.sort_values(by='Porcentaje', ascending=False)
display(resultado_df.head())
#display(df_eps.head())

,Cantidad de Registros,Porcentaje
Nombre de la entidad,,
NUEVA EPS S.A.,3213637,38.147962
ENTIDAD PROMOTORA DE SALUD SANITAS S.A.S.,940898,11.169072
SALUD TOTAL ENTIDAD PROMOTORA DE SALUD DEL REGIMEN CONTRIBUTIVO Y DEL REGIMEN SUBSIDIADO S.A.,888917,10.552023
EPS FAMISANAR S.A.S.,625881,7.429615
EPS SURAMERICANA S.A.,623137,7.397042


# 5.creación de BD SQLITE y tablas

In [ ]:
db_path="/content/drive/MyDrive/BigData/Dataset's/SQLITE/dbua.db"
db_dir =os.path.dirname(db_path)
if os.path.exists(db_dir):
  print(f"El directorio {db_dir} existe.")
else:
  os.makedirs(db_dir)
  print(f"El directorio {db_dir} fue creado.")

In [ ]:
try:
  conn=sqlite3.connect(db_path)
  cursor=conn.cursor()          #objeto que va a interactiar con la bd (dml/ddl)
  print("conectado")
except sqlite3.Error as e:
  print(f"Error al conectar a la base de datos: {e}")


## 5.1 creando tablas maestras

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Departamento (
    Id_Departamento INTEGER PRIMARY KEY AUTOINCREMENT,
    Nombre_Departamento TEXT UNIQUE NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Sisben (
    Id_Sisben INTEGER PRIMARY KEY AUTOINCREMENT,
    Nombre_Sisben TEXT UNIQUE NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Eps (
    codigo_Eps TEXT PRIMARY KEY,
    Nombre_Eps TEXT UNIQUE NOT NULL,
    regimen TEXT
)
''')

In [ ]:
# cursor.execute('''SELECT * FROM Departamento''')
# rows=cursor.fetchall()
# for rows in rows:
# print(row)

## 5.2 llenado de tablas maestras

In [ ]:
#funcion para cargar datos en tablas maestras (departamento, sisben, eps)
def cargar_datos_a_tablas_maestras(df,db_path,columna, tabla_destino):
  conn= sqlite3.connect(db_path)
  cursor=conn.cursor()
  valores_unicos =df[columna].unique()
  for valor in valores_unicos:
    if(pd.notna(valor)): #ignorar valores NaN
      try:
        cursor.execute(f"INSERT OR IGNORE INTO {tabla_destino} (Nombre_{tabla_destino.replace('_','')}) VALUES (?)", (valor,))
      except sqlite3.Error as e:
        print(f"Error al insertar en la tabla {tabla_destino}: {e}")
        pass
  conn.commit()
  conn.close()

In [ ]:
cargar_datos_a_tablas_maestras(df,db_path,"Departamento","Departamento")
cargar_datos_a_tablas_maestras(df,db_path,"Nivel del Sisbén","Sisben")

In [ ]:
def revisar_contenido_de_una_tabla(tabla_nombre, whereColumna='',whereValor='', limit=10, order_by_columna=None, order_asc=True):
  conn=sqlite3.connect(db_path)
  cursor=conn.cursor()
  cursor.execute("SELECT COUNT(*) FROM "+tabla_nombre)
  total_registros=cursor.fetchone()[0]
  print(f"Total de registros en la tabla {tabla_nombre}: {total_registros}")
  sql="SELECT * FROM "+tabla_nombre
  params = ()
  if len(whereColumna)>0:
    sql+=" WHERE "+whereColumna+"=?"
    params = (whereValor,)
  if(order_by_columna):
    orderbyTipo="ASC" if order_asc else "DESC"
    sql+=f" ORDER BY {order_by_columna} {orderbyTipo}"
  sql+=" LIMIT "+str(limit)
  print(sql)
  cursor.execute(sql, params)
  resultados=cursor.fetchall()
  for fila in resultados:
    print(fila)
  conn.close()

In [ ]:
revisar_contenido_de_una_tabla("Departamento",'','',100,'Id_departamento')

In [ ]:
revisar_contenido_de_una_tabla("Sisben",'','',100,'nombre_sisben')

## 5.3 tabla hija "municipio"

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Municipio (
    Id_Municipio INTEGER PRIMARY KEY AUTOINCREMENT,
    Nombre_Municipio TEXT NOT NULL,
    Id_Departamento INTEGER,
    FOREIGN KEY (Id_Departamento) REFERENCES Departamento(Id_Departamento),
    UNIQUE (Nombre_Municipio, Id_Departamento)
)
''')
conn.commit()
conn.close()

In [ ]:
#para cada municipio buscarle su departamento
def cargar_municipios(df,pd_path):
  conn= sqlite3.connect(db_path)
  cursor=conn.cursor()
  tot_creados=0
  for index,row in df.iterrows():
    departamento  =row['Departamento']
    municipio     =row['Municipio']
    cursor.execute("SELECT Id_Departamento FROM Departamento WHERE Nombre_Departamento=?", (departamento,))
    departamento_id=cursor.fetchone()
    if departamento_id:
      departamento_id=departamento_id[0]
      try:
        cursor.execute("INSERT OR IGNORE INTO Municipio (Nombre_Municipio, Id_Departamento) VALUES (?, ?)", (municipio, departamento_id))
        tot_creados+=1
        print(f"creado {tot_creados} => {municipio} en el dpto {departamento} ")
      except sqlite3.sqlite3.IntegrityError:
        pass
    else:
      print(f"ERRROR: Departamento {departamento} para el municipio {municipio} no encontrado")
  conn.commit()
  conn.close()

In [ ]:
cargar_municipios(df,db_path)

In [ ]:
revisar_contenido_de_una_tabla("Municipio",'','',10,'nombre_Municipio')

## 5.4 cargar EPS

In [ ]:
def cargar_datos_eps(df,db_path,regimen="Contributivo"):
 conn= sqlite3.connect(db_path)
 cursor=conn.cursor()
 total_creados=0
 valores_unicos = df[['Código de la entidad','Nombre de la entidad']].drop_duplicates()
 for index,row in valores_unicos.iterrows():
  codigo_eps= row['Código de la entidad']
  nombre_eps= row['Nombre de la entidad']
  try:
    cursor.execute("INSERT OR IGNORE INTO Eps (codigo_Eps, Nombre_Eps, Regimen) VALUES (?, ?,?)", (codigo_eps, nombre_eps,regimen))
    total_creados+=1
    print(f"creado {total_creados} => {nombre_eps} con el código {codigo_eps}")
  except sqlite3.IntegrityError as e:
    print(f"No fue posible crear la EPS {nombre_eps} con el código {codigo_eps} due to {e}")
    pass
 conn.commit()
 conn.close()

In [ ]:
cargar_datos_eps(df,db_path)

In [ ]:
revisar_contenido_de_una_tabla("Municipio",'','',10,'nombre_Municipio')

## 5.5 crear tabla nieta "detalle"
---


In [ ]:
conn= sqlite3.connect(db_path)
cursor=conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS Detalle (
    Id_Detalle INTEGER PRIMARY KEY AUTOINCREMENT,
    Id_Municipio INTEGER,
    Id_Sisben INTEGER,
    codigo_Eps TEXT,
    Genero TEXT,
    Grupo_etario TEXT,
    Tipo_afiliado TEXT,
    Estado_afiliado TEXT,
    Condicion_beneficiario TEXT,
    Regimen TEXT,
    Zona_afiliacion TEXT,
    Cantidad_registros INTEGER,
    FOREIGN KEY (Id_Municipio) REFERENCES Municipio(Id_Municipio),
    FOREIGN KEY (Id_Sisben) REFERENCES Sisben(Id_Sisben),
    FOREIGN KEY (codigo_Eps) REFERENCES Eps(codigo_Eps)
)
''')
conn.commit()
conn.close()

In [ ]:
def cargar_detalle(df,db_path):
  conn= sqlite3.connect(db_path)
  cursor=conn.cursor()
  total_creados=0
  for index,row in df.iterrows():
    municipio   =row['Municipio']
    departamento=row['Departamento']
    #--------para obtener el id del sisben
    if (row['Nivel del Sisbén'] is None):
      id_sisben=0
    else:
      sisben=row['Nivel del Sisbén']
      cursor.execute("SELECT Id_Sisben FROM Sisben WHERE Nombre_Sisben=?", (sisben,))
      sisben_id=cursor.fetchone()
      if sisben_id:
        id_sisben=sisben_id[0]
      else:
        print(f"ERRROR: Sisben {sisben} para el municipio {municipio} no encontrado")
    #----------para obtener id del municipio
    cursor.execute("SELECT Id_Municipio FROM Municipio WHERE Nombre_Municipio=? AND Id_Departamento=(SELECT Id_Departamento FROM Departamento WHERE Nombre_Departamento=?)", (municipio,departamento))
    municipio_id=cursor.fetchone()
    if municipio_id:
      municipio_id=municipio_id[0]
    else:
      print(f"ERRROR: Municipio {municipio} para el departamento {departamento} no encontrado")
    #--------------guarde cada registro en la tabla---
    try:
      cursor.execute('''
      INSERT INTO Detalle (Id_Municipio, Id_Sisben, codigo_Eps, Genero, Grupo_etario, Tipo_afiliado, Estado_afiliado, Condicion_beneficiario, Regimen, Zona_afiliacion, Cantidad_registros)
      VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                    (municipio_id,
                      id_sisben,
                      row['Código de la entidad'],
                      row['Género'],
                      row['Grupo etario'],
                      row['Tipo de afiliado'],
                      row['Estado del afiliado'],
                      row['Condición del beneficiario'],
                      row['Régimen'],
                      row['Zona de Afiliación'],
                      row['Cantidad de registros']
                    ))
      total_creados+=1
      print(f"creado {total_creados}")
    except sqlite3.IntegrityError as e:
      print(f"No fue posible crear el registro due to {e}")
      pass
  conn.commit()
  conn.close()



In [ ]:
cargar_detalle(df,db_path)

In [ ]:
revisar_contenido_de_una_tabla("Detalle",'','',10)

# 6.Integrar con tutelas radicadas

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
url_forder_csv="/content/drive/MyDrive/BigData/Dataset's/Tutelas_con_providencias"
listado_archivos= [f for f in os.listdir(url_forder_csv) if f.endswith(".csv")]
print(listado_archivos)
columnas_nombre=[
    "clasificacion",
    "num_expediente",
    "fecha_Radica",
    "demandante",
    "demandado",
    "primera_instancia",
    "segunda_instancia",
    "num_23Digitos",
    "providencia",
    "fechaSentencia"
]
df_tutelas=pd.DataFrame(columns=columnas_nombre)
display (df_tutelas)

In [ ]:
def cargar_data_desde_archivo_csv(ruta_archivo):
  try:
      df_temporal = pd.read_csv(ruta_archivo,sep=';',header=None,encoding='latin-1',on_bad_lines='skip')
      #agregarle nombre de columnas al df creado
      df_temporal.columns=columnas_nombre
      if (len(df_temporal.columns)==len(columnas_nombre)):
        print(f" archivo {os.path.basename(ruta_archivo)} cargada exitosamente")
        return df_temporal
      else:
        print(f"archivo no trabajado {os.path.basename(ruta_archivo)} no tiene 10 columnas")
        return None

  except Exception as e:
      print(f"Error al procesar el archivo {os.path.basename(ruta_archivo)}: {str(e)}")
      return None

In [ ]:
df_list=[]  #lista de df resultante de los datos de cada uno de los CSV
with ThreadPoolExecutor() as executor:
  future_to_file= {executor.submit(cargar_data_desde_archivo_csv,os.path.join(url_forder_csv,archivo_nombre)):archivo_nombre for archivo_nombre in listado_archivos}
  for future in as_completed(future_to_file):
    df=future.result()
    if df is not None:
      df_list.append(df)
#--------una vez termine de cargar todos los CSV combinelos en el dataframa df_tutelas
if df_list:
  df_tutelas= pd.concat(df_list,ignore_index=True)
  # reemplazar las palabras "NULL" por none
  df_tutelas.replace("NULL",None,inplace=True)
  display(df_tutelas.head())


In [ ]:
df_tutelas.info()

## 6.1 extraer los expedientes donde los demandados "eps"

In [ ]:
df_tutelas_eps=df_tutelas[df_tutelas['demandado'].str.contains("EPS",na=False)]
display(df_tutelas_eps.head())
df_tutelas_eps.count()

## 6.2 Extraer las EPS dbua.db (sqlite)

In [ ]:
conn= sqlite3.connect(db_path)
df_eps=pd.read_sql_query("SELECT * FROM Eps",conn)
conn.close()
display(df_eps.head())

## 6.3 contar las tutelas en contra de cada EPS (df_eps)

In [ ]:
df_eps['total_tutelas']=0
demandados_encontrados =set()
for index,row in df_eps.iterrows():
  eps_nombre=row['Nombre_Eps']
  matching_tutelas = df_tutelas_eps[df_tutelas_eps['demandado'].str.contains(eps_nombre,na=False,case=False)]  #mejorar el código "NUEVA EPS" *reglas para mejorar las busquedas
  total_tutelas=matching_tutelas.shape[0]
  #df_eps.at[index,'total_tutelas']=total_tutelas
  df_eps.loc[index,'total_tutelas']=total_tutelas
  demandados_encontrados.update(matching_tutelas['demandado'].tolist())

demandados_no_encontrados=set(df_tutelas_eps['demandado'].unique())-demandados_encontrados
print(f"EPS demandadas encontrados: {len(demandados_encontrados)}")
print(f"EPS demandadas NO encontradas: {len(demandados_no_encontrados)}")
for demandado in demandados_no_encontrados:
  print(demandado)

In [ ]:
display(df_eps.head())

# ....MEJORAR EL CÓDIGO para que encuentre las diversas variaciones de los nombres de las EPS